In [20]:
import csv
import pandas as pd
import random as rd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
import re

In [3]:
file_path = r'C:\\Users\\user\\Desktop\\hansol\\3rd_project\\NLP_new_\\NLP-project\\NLP project\\crawling_data\\(new)korean1-317.csv'

with open(file_path, 'r', encoding='utf-8') as file :
    raw_cv = csv.reader(file)
    c = list(raw_cv)

In [4]:
cover_letter = pd.DataFrame(data = c, columns=['text'])
cover_letter['합격여부']='합격'
cover_letter.loc[178:, '합격여부'] = '불합격'

In [5]:
star1n2 = cover_letter.iloc[-76:] # 평점 2점까지( 갯수 균형 맞추기 )
star4n5 = cover_letter.iloc[:77] # 평점 4점까지
# star1n2 = cover_letter.iloc[-142:] # 평점 2점까지

In [13]:
rd.seed(123)
pick_n = rd.sample(range(176,302), 62) # 2점 자소서 62개의 행 번호 추출
star1 = cover_letter.iloc[-15:]

star2 = []
for i, row in cover_letter.iterrows() :
    if i in pick_n:
        star2.append(row['text'])

print(star2)

['00년 차 JAVA 개발자 000입니다. 백 엔드는 Java, Spring Framework, Mybatis를 프론트 엔드는 JSP를 사용합니다. DB는 오라클, mysql을 사용하여 웹 개발 및 서버 점검 및 솔루션 유지보수 일을 하고 있습니다 OO고객사 2차 인증 개발 사업에 참여해서 2차 인증 RESTful API 설계 및 고객의 요청사항을 접수하여서 향후 개발 방향에 반영해서 2차 인증 개발을 완료했습니다. OO고객사 시스템 재구축 사업에 2명으로 구성된 팀에서 메인 개발자로 참여하였고 RESTful API 설계, 리눅스 서버에 앱 배포 및 서비스 모니터링 업무를 수행했으며 안정적으로 시스템을 구축하여 오픈하였습니다. 매일 수많은 양의 뉴스가 보도되고 가짜뉴스가 판을 치다 보니 속보라고 할지라도 색안경은 끼고 보는 일이 많아졌습니다. 저와 같은 사람들이 많아져서 그런지 팩트체크를 하는 사이트가 나오기 시작했으며 JTBC 뉴스에서도 사실 여부가 확인된 뉴스를 표기해주는 서비스가 있으면 좋을 것 같습니다. 예를 들어 유튜브의 인증된 사용자임을 표기하는 체크 아이콘이 아이디 옆에 표기되는 것처럼 사실 여부가 확인되면 뉴스 제목 앞에 체크 표시를 두어서 사실 여부가 확인된 사항임을 사용자에게 알려줌으로써 사용자는 해당 뉴스를 믿고 볼 수 있으며 해당 서비스를 통해 타 언론사와 차별화된 서비스를 제공함으로써 보다 더 경쟁력 있는 JTBC가 될 수 있다고 생각합니다   ', '"부모님과 군 전산병 생활" 부모님은 늘 인생의 길잡이이자, 삶의 멘토 역할을 해주셨습니다. 직업에 대한 자부심을 가지신 아버지는 매사에 최선을 다하는 자세를 강조해 오셨습니다. 아버지의 가르침은 제가 작은 일이라도 세심하게 신경 쓰고, 노력할 수 있게 만들어 주었습니다. 이는 저의 \'노력하는 사람은 못 할 일이 없다.’라는 가치관 형성에 영향을 주었습니다. 또,“끝까지 할 수 있어?”라는 말과 함께 저를 지지해 주시던 어머니는 맡은 일을 중간에 포기하지 않는 끈기를 길러 주셨습니다. 

In [14]:
star2_df = pd.DataFrame(star2, columns=['text'])
star2_df['합격여부']='불합격'

star1 = star1.reset_index(drop=True)
star2_df = star2_df.reset_index(drop=True)

star1n2 = pd.concat([star1,star2_df])
star1n2

,text,합격여부
0,가장 어려운 질문인데 국민건강을 보장하고 국민보험심사를 평가하는 평가원이 아닌가 싶...,불합격
1,시대를 거스르는 개발 및 발명만이 우선이 아닌 사소한 것의 자그마한 변화 또한 혁신...,불합격
2,"""하스스톤"" 월드오브 워크래프트의 세계관에서 파생된 게임으로 CCG형태이다. 기존의...",불합격
3,"'좋아하고 즐길 때, 때로는 짜증나고 화날 때' 어떤 것을 수행함에 있어서 좋아하고...",불합격
4,"""The Best Value Partner"" 현대정보기술에 지원한 이유는 세 가지입...",불합격
...,...,...
57,"평소에 삼성 제품을 애용하면서 삼성의 튼실한 H/W와 S/W를 경험하며, 언젠가 발...",불합격
58,[성장과정] 저는 꿈을 향한 특별한 `열정`이 있습니다. 아버지께서 운영하시던 컴퓨...,불합격
59,"사람들은 버스, 비행기 등 운송수단을 통해 새로운 경험과 시작을 하거나 이 전과의 ...",불합격
60,저는 도리와 의리를 지키는 삶을 살고 있습니다. 항상 어떠한 상황에서도 타인에게 신...,불합격


In [15]:

star12n45 = pd.concat([star1n2,star4n5])

print(star12n45)

                                                 text 합격여부
0   가장 어려운 질문인데 국민건강을 보장하고 국민보험심사를 평가하는 평가원이 아닌가 싶...  불합격
1   시대를 거스르는 개발 및 발명만이 우선이 아닌 사소한 것의 자그마한 변화 또한 혁신...  불합격
2   "하스스톤" 월드오브 워크래프트의 세계관에서 파생된 게임으로 CCG형태이다. 기존의...  불합격
3   '좋아하고 즐길 때, 때로는 짜증나고 화날 때' 어떤 것을 수행함에 있어서 좋아하고...  불합격
4   "The Best Value Partner" 현대정보기술에 지원한 이유는 세 가지입...  불합격
..                                                ...  ...
72  [세계의 Standard Bank] 학부시절 HW, SW, Network 라는 다양...   합격
73  [외부와 소통하는 사람] 저는 배려심이 많고 화합 능력이 뛰어나 팀 사람들을 편안한...   합격
74  종합설계프로젝트를 이수하며 학부생들과 ‘오픈콥터’라는 프로젝트를 진행했었습니다. 오...   합격
75  상상하는 것은 곧 현실이 되는 이 시대에 IT분야야말로 앞으로의 시대에서 가장 큰 ...   합격
76  저에게 맡겨진 일에 대해 주도적으로 할 일을 찾으며 책임감을 다 할 수 있는 사람입...   합격

[154 rows x 2 columns]


In [16]:
texts = star12n45['text'].astype(str).tolist() # 평점이 12 vs 45
labels = star12n45['합격여부'].tolist()

In [17]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(texts, encoded_labels, test_size=0.2, random_state=123)

In [21]:
def nr_text(texts):
    n_min = 2
    corpus = []
    for a_line in texts :
        pre = re.sub(r'\W', ' ', a_line)
        pre = re.sub(r'_', ' ', pre)
        pre = re.sub(r'\d+','', pre) 
        pre = nltk.word_tokenize(pre)
        pre = [x for x in pre if len(x) >= n_min]
        corpus += pre
    return corpus

tkned_text = nr_text(texts)
tkned_text

['가장',
 '어려운',
 '질문인데',
 '국민건강을',
 '보장하고',
 '국민보험심사를',
 '평가하는',
 '평가원이',
 '아닌가',
 '싶습니다',
 '국민을',
 '배려하고',
 '국민의',
 '건강을',
 '증진하는',
 '일이라면',
 '나보다',
 '남의',
 '생각하는',
 '우선이',
 '아닌가',
 '싶습니다',
 '본인이',
 '적합한지',
 '한지는',
 '일하면',
 '있지',
 '않을',
 '생각합니다',
 '뽑아',
 '주십시오',
 '열심히',
 '하겠습니다',
 '제가',
 '가진',
 '역량은',
 '작지만',
 '제가',
 '있는',
 '부분은',
 '결코',
 '작다고',
 '생각하지',
 '않습니다',
 '기존의',
 '방식이',
 '복잡하다거나',
 '일이',
 '많아',
 '진다면',
 '소프트웨어적인',
 '편의를',
 '이용해서',
 '단순하게나',
 '빨리',
 '대응할',
 '방법을',
 '제시하거나',
 '구현',
 '해도',
 '되는',
 '부분에',
 '대해서는',
 '엔지니어와',
 '요구하는',
 '갑과의',
 '의견을',
 '조율해서',
 '합니다',
 '대개는',
 '받아들여지지',
 '않는',
 '부분이',
 '있었지만',
 '결국엔',
 '쉬운',
 '방향으로',
 '개발하자고',
 '의견을',
 '수용해',
 '주셨습니다',
 '여러',
 '프로그래머와',
 '회사가',
 '함께',
 '하는',
 '프로젝트라',
 '나의',
 '일이',
 '여러',
 '분의',
 '눈치를',
 '보는',
 '일이여서',
 '두드러지진',
 '않았지만',
 '일이',
 '마무리',
 '되고',
 '나면',
 '기쁘고',
 '보람을',
 '느꼈습니다',
 '업무수행에',
 '필요한',
 '정보는',
 '동료의',
 '도움이나',
 '갑의',
 '관련',
 '자료를',
 '수집하거나',
 '관련',
 '엔지니어의',
 '조언을',
 '들어',
 '업무를',
 '수행했습니다',
 '기업의',
 '업무를',
 '

In [22]:
# TaggedDocument 생성
tagged_data = [TaggedDocument(words=nouns, tags=[str(i)]) for i, nouns in enumerate(tkned_text)]

# Doc2Vec 모델 학습
doc2vec_model = Doc2Vec(vector_size=50, window=2, min_count=1, workers=4, epochs=100)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [11]:
# doc2vec_model = Doc2Vec(
#     vector_size=100,  # 벡터 차원 수
#     window=5,         # 컨텍스트 윈도우 크기
#     min_count=2,      # 최소 출현 빈도
#     workers=4,        # 사용 스레드 수
#     epochs=40,        # 학습 반복 횟수
#     dm=1              # DM 모델 사용
# )
# doc2vec_model.build_vocab(tagged_data)
# doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train_vectors = [doc2vec_model.infer_vector(word_tokenize(doc.lower())) for doc in X_train]
X_test_vectors = [doc2vec_model.infer_vector(word_tokenize(doc.lower())) for doc in X_test]

classifier = RandomForestClassifier(n_estimators=100, random_state=123)
classifier.fit(X_train_vectors, y_train)

y_pred = classifier.predict(X_test_vectors)

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         불합격       0.71      0.71      0.71        14
          합격       0.76      0.76      0.76        17

    accuracy                           0.74        31
   macro avg       0.74      0.74      0.74        31
weighted avg       0.74      0.74      0.74        31



In [25]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state=123)

param_grid = {
    'n_estimators': [100, 200, 300], # 생성할 트리 개수
    'max_depth': [10, 20, 30], # 각 트리의 최대 깊이
    'min_samples_split': [2, 5, 10], # 내부 노드 분할 시 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4], # 리프 노드 최소 샘플 수
    'max_features': ['sqrt', 'log2'] #  각 트리를 분할할 때 고려할 최대 특징 수입니다. 제곱근(sqrt)과 로그(log2) 값 사용
}

grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=3,  # 교차 검증 3번
                           n_jobs=-1,  # 가능한 모든 프로세서를 사용하여 병렬 처리를 수행
                           verbose=2)  # 진행 상황을 출력

grid_search.fit(X_train_vectors, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_vectors)

print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Best parameters found:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.59      0.71      0.65        14
           1       0.71      0.59      0.65        17

    accuracy                           0.65        31
   macro avg       0.65      0.65      0.65        31
weighted avg       0.66      0.65      0.65        31



In [26]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rf = RandomForestClassifier(random_state=123)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=3, n_jobs=-1, random_state=123, verbose=2)

random_search.fit(X_train_vectors, y_train)

print("Best parameters found: ", random_search.best_params_)

best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test_vectors)

print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found:  {'max_depth': 21, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 3, 'n_estimators': 143}
              precision    recall  f1-score   support

           0       0.62      0.71      0.67        14
           1       0.73      0.65      0.69        17

    accuracy                           0.68        31
   macro avg       0.68      0.68      0.68        31
weighted avg       0.68      0.68      0.68        31



In [30]:
from sklearn.svm import SVC # SVM

svm_cf = SVC(kernel='linear', random_state=123)
svm_cf.fit(X_train_vectors, y_train)

y_pred_svm = svm_cf.predict(X_test_vectors)
print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         불합격       0.57      0.57      0.57        14
          합격       0.65      0.65      0.65        17

    accuracy                           0.61        31
   macro avg       0.61      0.61      0.61        31
weighted avg       0.61      0.61      0.61        31



In [32]:
from sklearn.ensemble import VotingClassifier # 앙상블

voting_classifier = VotingClassifier(estimators=[
    ('rf', classifier),
    ('svc', svm_cf)
], voting='hard')

voting_classifier.fit(X_train_vectors, y_train)
y_pred_voting = voting_classifier.predict(X_test_vectors)
print(classification_report(y_test, y_pred_voting, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         불합격       0.61      0.79      0.69        14
          합격       0.77      0.59      0.67        17

    accuracy                           0.68        31
   macro avg       0.69      0.69      0.68        31
weighted avg       0.70      0.68      0.68        31

